1. Get crv emissions per pool.
2. Get fees claimed per pool.
3. divide the two to get swap revenue per crv emitted per gauge.

In [1]:
# subgraph querying modules
import datetime
import calendar

In [2]:
# plotting tools
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# brownie
import os
import sys
sys.path.append("../../")

import brownie

from utils.contract_utils import init_contract
from utils.network_utils import configure_network_and_connect
from utils.eth_blocks_utils import get_block_for_timestamp


In [4]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
configure_network_and_connect(
    node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
    network_name='mainnet'
)

# Pools and Gauges

Get dict of pools (keys) and gauge (value)

In [5]:
ethereum_registry_contract = init_contract("0x90E00ACe148ca3b23Ac1bC8C240C2a7Dd9c2d7f5")
ethereum_registry_contract.info()

  @title Curve Registry
  @author Curve.Fi



Get pools and gauges:

In [6]:
pool_count = ethereum_registry_contract.pool_count()
eth_pools_and_gauges = {}
for pool_id in range(pool_count):

    pool_addr = ethereum_registry_contract.pool_list(pool_id)
    gauge_addr = ethereum_registry_contract.get_gauges(pool_addr)
    lp_token_addr = ethereum_registry_contract.get_lp_token(pool_addr)

    # get lp_token name:
    pool_token_contract = init_contract(lp_token_addr)
    pool_token_name = pool_token_contract.name()

    eth_pools_and_gauges[pool_addr] = [gauge_addr[0][0], lp_token_addr, pool_token_name]

eth_pools_and_gauges

{'0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7': ['0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A',
  '0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490',
  'Curve.fi DAI/USDC/USDT'],
 '0xDeBF20617708857ebe4F679508E7b7863a8A8EeE': ['0xd662908ADA2Ea1916B3318327A97eB18aD588b5d',
  '0xFd2a8fA60Abd58Efe3EeE34dd494cD491dC14900',
  'Curve.fi aDAI/aUSDC/aUSDT'],
 '0xA96A65c051bF88B4095Ee1f2451C2A9d43F53Ae2': ['0x6d10ed2cF043E6fcf51A0e7b4C2Af3Fa06695707',
  '0xaA17A236F2bAdc98DDc0Cf999AbB47D47Fc0A6Cf',
  'Curve.fi ETH/aETH'],
 '0x79a8C46DeA5aDa233ABaFFD40F3A0A2B1e5A4F27': ['0x69Fb7c45726cfE2baDeE8317005d3F94bE838840',
  '0x3B3Ac5386837Dc563660FB6a0937DFAa5924333B',
  'Curve.fi yDAI/yUSDC/yUSDT/yBUSD'],
 '0xA2B47E3D5c44877cca798226B7B8118F9BFb7A56': ['0x7ca5b0a2910B33e9759DC7dDB0413949071D7575',
  '0x845838DF265Dcd2c412A1Dc9e959c7d08537f8a2',
  'Curve.fi cDAI/cUSDC'],
 '0x0Ce6a5fF5217e38315f87032CF90686C96627CAA': ['0x90Bb609649E0451E5aD952683D64BD2d1f245840',
  '0x194eBd173F6cDacE046C53eACcE9B953F28411d1',

# CRV emissions per pool:

In [7]:
crv_token_addr = "0xD533a949740bb3306d119CC777fa900bA034cd52"
crv_token = init_contract(crv_token_addr)
crv_token.info()

  @title Curve DAO Token
  @notice ERC20 with piecewise-linear mining supply.
  @author Curve Finance
  @details Based on the ERC-20 token standard as defined
           at https://eips.ethereum.org/EIPS/eip-20



CRV emissions rate per block

In [8]:
crv_token.rate() * 1e-18

7.327853447857532

get number of blocks between two timestamps:

In [9]:
block_time_start = int(
    calendar.timegm((datetime.datetime.now() - datetime.timedelta(days=7)).date().timetuple())
)
block_time_end = int(
    calendar.timegm((datetime.datetime.now()).date().timetuple())
)
crv_token.mintable_in_timeframe(block_time_start, block_time_end) * 1e-18

4431885.7652642345

get gauge weights from registry for block:

In [10]:
gauge_controller_address = "0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB"
gauge_controller = init_contract(gauge_controller_address)
gauge_controller.info()

  @title Gauge Controller
  @notice Controls liquidity gauges and the issuance of
          coins through the gauges
  @author Curve Finance



Get crv emitted for gauge

In [11]:
time_start = int(datetime.datetime.strptime("2021-08-01", "%Y-%m-%d").timestamp())
time_end = int(datetime.datetime.strptime("2021-11-01", "%Y-%m-%d").timestamp())
delta_time = int(datetime.timedelta(days=1).total_seconds())
date_range = list(range(time_start,time_end,delta_time))

In [12]:
emissions_data = pd.DataFrame()
for timestamp in date_range:

    block_number = get_block_for_timestamp(timestamp=timestamp)

    with brownie.multicall(block_identifier=block_number):

        for pool_addr, pool_details in eth_pools_and_gauges.items():

            pool_data = {}

            gauge_addr = pool_details[0]
            lp_token_addr = pool_details[1]
            pool_name = pool_details[2]

            # get crv emissions for timestamp from the past day:
            crv_minted = crv_token.mintable_in_timeframe(
                int(timestamp - delta_time),
                timestamp
            ) * 1e-18

            # get gauge relative weight
            gauge_relative_weight = gauge_controller.gauge_relative_weight(gauge_addr) * 1e-18

            # get virtual price:
            try:
                lp_token_virtual_price = (
                    ethereum_registry_contract.get_virtual_price_from_lp_token(lp_token_addr)
                ) * 1e-18
            except TypeError:
                lp_token = init_contract(lp_token_addr)
                lp_token_virtual_price = lp_token.get_virtual_price() * 1e-18

            # get total supply of lp_token:
            lp_token = init_contract(lp_token_addr)
            total_supply_lp_token = lp_token.totalSupply() * 1e-18

            # pool data
            pool_data['timestamp'] = timestamp
            pool_data['block_number'] = block_number
            pool_data['pool_name'] = pool_name
            pool_data['pool_addr'] = pool_addr
            pool_data['lp_token_addr'] = lp_token_addr
            pool_data['gauge_addr'] = gauge_addr
            pool_data['gauge_relative_weight'] = gauge_relative_weight
            pool_data['lp_token_virtual_price'] = lp_token_virtual_price
            pool_data['total_supply_lp_token'] = total_supply_lp_token
            pool_data['crv_minted_past_1d'] = crv_minted
            pool_data['crv_to_gauge'] = gauge_relative_weight * crv_minted

            pool_data = pd.DataFrame(
                data=pool_data,
                index=[pd.to_datetime(pool_data['timestamp'], unit='s')]
            )

            # join df
            emissions_data = pd.concat([emissions_data, pool_data])


In [13]:
emissions_data

,timestamp,block_number,pool_name,pool_addr,lp_token_addr,gauge_addr,gauge_relative_weight,lp_token_virtual_price,total_supply_lp_token,crv_minted_past_1d,crv_to_gauge
2021-07-31 22:00:00,1627768800,12935810,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.030036,1.018249,1.387837e+09,752918.583562,22614.480396
2021-07-31 22:00:00,1627768800,12935810,Curve.fi aDAI/aUSDC/aUSDT,0xDeBF20617708857ebe4F679508E7b7863a8A8EeE,0xFd2a8fA60Abd58Efe3EeE34dd494cD491dC14900,0xd662908ADA2Ea1916B3318327A97eB18aD588b5d,0.013610,1.059715,1.583507e+08,752918.583562,10246.937315
2021-07-31 22:00:00,1627768800,12935810,Curve.fi ETH/aETH,0xA96A65c051bF88B4095Ee1f2451C2A9d43F53Ae2,0xaA17A236F2bAdc98DDc0Cf999AbB47D47Fc0A6Cf,0x6d10ed2cF043E6fcf51A0e7b4C2Af3Fa06695707,0.005305,1.019913,2.031894e+04,752918.583562,3994.574615
2021-07-31 22:00:00,1627768800,12935810,Curve.fi yDAI/yUSDC/yUSDT/yBUSD,0x79a8C46DeA5aDa233ABaFFD40F3A0A2B1e5A4F27,0x3B3Ac5386837Dc563660FB6a0937DFAa5924333B,0x69Fb7c45726cfE2baDeE8317005d3F94bE838840,0.003551,1.118360,2.548384e+07,752918.583562,2673.676793
2021-07-31 22:00:00,1627768800,12935810,Curve.fi cDAI/cUSDC,0xA2B47E3D5c44877cca798226B7B8118F9BFb7A56,0x845838DF265Dcd2c412A1Dc9e959c7d08537f8a2,0x7ca5b0a2910B33e9759DC7dDB0413949071D7575,0.011510,1.080587,1.350679e+08,752918.583562,8665.791993
...,...,...,...,...,...,...,...,...,...,...,...
2021-10-31 22:00:00,1635717600,13527346,Curve.fi Factory USD Metapool: Alchemix USD,0x43b4FdFD4Ff969587185cDB6f0BD875c5Fc83f8c,0x43b4FdFD4Ff969587185cDB6f0BD875c5Fc83f8c,0x9582C4ADACB3BCE56Fea3e590F05c3ca2fb9C477,0.028920,1.007269,4.964252e+08,633126.537895,18310.020018
2021-10-31 22:00:00,1635717600,13527346,Curve.fi USD-BTC-ETH,0x80466c64868E1ab14a1Ddf27A676C3fcBE638Fe5,0xcA3d75aC011BF5aD07a98d02f18225F9bD9A6BDF,0x6955a55416a06839309018A8B0cB72c4DDC11f15,0.001784,1.011439,7.285534e+03,633126.537895,1129.735870
2021-10-31 22:00:00,1635717600,13527346,Curve.fi USD-BTC-ETH,0xD51a44d3FaE010294C616388b506AcdA1bfAAE46,0xc4AD29ba4B3c580e6D59105FFf484999997675Ff,0xDeFd8FdD20e0f34115C7018CCfb655796F6B2168,0.078399,1.005494,5.118408e+05,633126.537895,49636.505938
2021-10-31 22:00:00,1635717600,13527346,Curve.fi Factory USD Metapool: Magic Internet ...,0x5a6A4D54456819380173272A5E8E9B9904BdF41B,0x5a6A4D54456819380173272A5E8E9B9904BdF41B,0xd8b712d29381748dB89c36BCa0138d7c75866ddF,0.251976,1.002929,2.488265e+09,633126.537895,159532.859485


# swap fees to liquidity providers per pool per week:

In [50]:
emissions_and_revenue = pd.DataFrame()
for pool_addr in emissions_data.pool_addr.unique():

    pool_data = emissions_data[emissions_data.pool_addr == pool_addr]
    pool_data = pool_data.sort_values(by="block_number", ascending=True)

    pool_data['virtual_price_diff'] = pool_data.lp_token_virtual_price.diff()
    pool_data['swap_fee_revenue'] = pool_data.virtual_price_diff * pool_data.total_supply_lp_token * 2
    pool_data['swap_revenue_to_crv_emissions_ratio'] = pool_data.swap_fee_revenue / pool_data.crv_to_gauge

    emissions_and_revenue = pd.concat([emissions_and_revenue, pool_data])

save data:

In [53]:
emissions_and_revenue.to_csv(f"./emissions_and_revenue_{time_start.strftime('%Y%m%d')}_{time_end.strftime('%Y%m%d')}.csv")